<a href="https://colab.research.google.com/github/RTae/Data-science-essentials/blob/master/week2/DSE_Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

##Pipeline
1. get data
2. extract feature
3. run model
4. test model
5. deploy model

In [ ]:
!pip install botnoi==0.2.1

     |████████████████████████████████| 16.0MB 292kB/s 


In [ ]:
from botnoi import scrape as sc
from botnoi import cv
from tqdm import tqdm
import os

In [ ]:
def extractimagefeat(query):
  foldername = 'images/'+query
  isdir = os.path.isdir(foldername) 
  if not isdir:
    os.makedirs(foldername)
  #get images from google search
  imglist = sc.get_image_urls(query)
  i = 1
  for img in tqdm(imglist[0:30], position=0, leave=True):
    #extract image features from each images and save to files
    try:
      savepath = foldername + '/' + str(i)+'.p'
      a = cv.image(img)
      a.getresnet50()
      a.save(savepath)
    except:
      pass
  return 'complete'


In [ ]:
import glob
import pandas as pd
import pickle
def createdataset(imgfolder):
  #imgfolder = glob.glob('images/*')
  dataset = []
  for cls in imgfolder:
    clsset = pd.DataFrame()
    pList = glob.glob(cls+'/*')
    featvec = []
    for p in pList:
      dat = pickle.load(open(p,'rb'))
      featvec.append(dat.getresnet50())
    #featvec = np.vstack(featvec)
    clsset['feature'] = featvec
    cls = cls.split('/')[-1]
    clsset['label'] = cls
    dataset.append(clsset)
  return pd.concat(dataset,axis=0)

## Train model

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
def trainmodel(dataset,modfile=''):
  trainfeat, testfeat, trainlabel, testlabel = train_test_split(dataset['feature'], dataset['label'], test_size=0.33, random_state=42)
  clf = LinearSVC()
  clf = CalibratedClassifierCV(clf) 
  mod = clf.fit(np.vstack(trainfeat.values),trainlabel.values)
  res = mod.predict(np.vstack(testfeat.values))
  if modfile!='':
    pickle.dump(mod,open(modfile,'wb'))
  acc = sum(res == testlabel.values)/len(res)
  return mod,acc

In [ ]:
#pipeline to create model
clsList = ['หมา','แมว']
for c in clsList:
  extractimagefeat(c)
imgfolder = ['images/'+c for c in clsList]
dataset = createdataset(imgfolder)
#dataset
modFile = 'mymodel.p'
mod,acc = trainmodel(dataset,modFile)

complete: 0.00%


  0%|          | 0/30 [00:00<?, ?it/s]

completed


100%|██████████| 30/30 [00:30<00:00,  1.01s/it]


complete: 0.00%


  0%|          | 0/30 [00:00<?, ?it/s]

completed


100%|██████████| 30/30 [00:50<00:00,  1.69s/it]


In [ ]:
# output function
mymod = pickle.load(open(modFile,'rb'))
def predictimg(imgurl):
  a = cv.image(imgurl)
  feat = a.getresnet50()
  probList = mod.predict_proba([feat])[0]
  maxprobind = np.argmax(probList)
  prob = probList[maxprobind]
  outclass = mymod.classes_[maxprobind]
  result = {}
  result['class'] = outclass
  result['probability'] = prob
  return result


In [ ]:
a=predictimg('https://do.lnwfile.com/oddu08.jpg')
a

{'class': 'หมา', 'probability': 0.6391101356080122}

In [ ]:
predictimg('https://cf.shopee.co.th/file/f338f8d43a8c609c9518d8a4a9915947')

{'class': 'หมา', 'probability': 0.6446518233360707}

In [ ]:
url = 'https://ichef.bbci.co.uk/news/640/cpsprodpb/51F3/production/_106997902_gettyimages-611696954.jpg'
predictimg(url)

{'class': 'แมว', 'probability': 0.9047286301151807}